## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-22-13-00-00 +0000,missionlocal,Crane Cove coyote,https://missionlocal.org/2025/08/sf-photos-cra...
1,2025-08-22-12-58-19 +0000,nyt,"Frank Caprio, Kind Judge on Rhode Island TV, D...",https://www.nytimes.com/2025/08/21/us/judge-fr...
2,2025-08-22-12-58-12 +0000,nyt,"F.B.I. Searches Maryland Home of John Bolton, ...",https://www.nytimes.com/2025/08/22/us/politics...
3,2025-08-22-12-56-46 +0000,nyt,‘No Meeting Planned’ Between Putin and Zelensk...,https://www.nytimes.com/2025/08/22/world/europ...
4,2025-08-22-12-55-51 +0000,nyt,Gaza City and Surrounding Areas Officially Hit...,https://www.nytimes.com/2025/08/22/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,61
93,new,23
192,ukraine,16
177,out,16
27,gaza,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
26,2025-08-22-11-36-52 +0000,nypost,Ukraine obliterates critical part of Russia’s ...,https://nypost.com/2025/08/22/world-news/ukrai...,120
231,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...,119
248,2025-08-21-19-17-34 +0000,nyt,NY Appeals Court Throws Out Trump’s Half-Billi...,https://www.nytimes.com/2025/08/21/nyregion/tr...,115
189,2025-08-21-23-19-00 +0000,wsj,"Alina Habba, Trump’s Top Prosecutor in New Jer...",https://www.wsj.com/us-news/law/alina-habba-ne...,113
131,2025-08-22-03-52-00 +0000,wsj,A federal judge has ordered that the governmen...,https://www.wsj.com/politics/policy/federal-ju...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
26,120,2025-08-22-11-36-52 +0000,nypost,Ukraine obliterates critical part of Russia’s ...,https://nypost.com/2025/08/22/world-news/ukrai...
184,73,2025-08-21-23-43-37 +0000,nypost,California Gov. Gavin Newsom signs redistricti...,https://nypost.com/2025/08/21/us-news/californ...
231,58,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...
126,55,2025-08-22-04-36-24 +0000,wapo,"Erik Menendez, who was convicted of killing hi...",https://www.washingtonpost.com/nation/2025/08/...
30,54,2025-08-22-11-15-23 +0000,bbc,"Famine confirmed in Gaza City for first time, ...",https://www.bbc.com/news/articles/c05ed5rgld3o...
110,53,2025-08-22-06-30-55 +0000,nypost,Florida nurse Alexis Von Yates takes plea deal...,https://nypost.com/2025/08/22/us-news/florida-...
131,51,2025-08-22-03-52-00 +0000,wsj,A federal judge has ordered that the governmen...,https://www.wsj.com/politics/policy/federal-ju...
212,46,2025-08-21-22-03-00 +0000,wsj,Two Fed officials offered competing assessment...,https://www.wsj.com/economy/central-banking/fe...
18,43,2025-08-22-12-05-57 +0000,wapo,He agreed to conduct a band when he turned 100...,https://www.washingtonpost.com/lifestyle/2025/...
44,41,2025-08-22-10-45-16 +0000,nypost,Israel warns Hamas that ‘gates of hell’ will o...,https://nypost.com/2025/08/22/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
